Log in to facebook?



In [7]:

var enterFacebook = () => {
    console.log('Facebook: Sign in required');
    var credentials = getCredentials('facebook.com');
    return client.click('input[name*="email"]')
    .keys(credentials.username)
    .pause(1000)
    .then(() => console.log('Facebook: Require password'))
    .click('input[name*="pass"]')
    .keys(credentials.password)
    .submitForm('[type="submit"]')
    .pause(2000)
    .isExisting('.cp-challenge-form')
    .then(is => {
        if(is) {
            throw new Error('captcha');
        }
    });
}

var loginFacebook = () => {
    return client
    .getUrl()
    .then(url => {
        var loggedIn = url.indexOf('facebook') > -1 && url.indexOf('login') == -1;
        return loggedIn
            ? client
                .isVisible('input[name*="email"]')
                .then(is => is ? enterFacebook() : client)
            : client.url('https://www.facebook.com/')
                .isVisible('input[name*="email"]')
                .then(is => is ? enterFacebook() : client)
    })
}
module.exports = loginFacebook;
loginFacebook;



[Function: loginFacebook]

download Facebook messages?



In [ ]:
var scrollFacebookThreads = (threads) => {
    return client
    .execute(() => {
        document.getElementsByClassName(
            'scrollable')[0]
            .scrollTop += 10000;
    })
    .pause(2000)
    .then(() => getFacebookThreads(threads))
};

var getFacebookThreads = (threads) => {
    return client    
    .execute(() => {
        var iterator = document.evaluate(
            '//*[contains(@data-href, "messages")]/@data-href',
            document, null,
            XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
        var co = []; var m;
        while ((m = iterator.iterateNext()) && co.push(m.value));
        return co;
    })
    .then(els => {
        var newThreads = els.value.filter(e => threads.indexOf(e) === -1);
        threads = threads.concat(newThreads);
        return newThreads.length > 0
            ? scrollFacebookThreads(threads)
            : Promise.resolve(threads);
    })
};

var listFacebookThreads = (threads) => {
    return client.getUrl()
    .then(url => url.indexOf('/messages/t') == -1
        ? client
              .click('[data-tooltip-content="Messages"]')
              .click('a[href*="/messages/t"]')
        : Promise.resolve([]))
    .pause(2000)
    .then(() => getFacebookThreads(threads || []))
    .then(r => threads = r)
    .then(url => url.indexOf('/messages/archived') == -1
        ? client.url('https://www.facebook.com/messages/archived')
              .pause(1000)
              .click('.scrollable a[href=""]')
        : Promise.resolve([]))
    .pause(2000)
    .then(() => getFacebookThreads(threads || []))
};
module.exports = listFacebookThreads;
listFacebookThreads;



Get messages from facebook thread?



In [8]:
var importer = require('../Core');
var moment = require('moment');
var chrono = require('chrono-node');

var getThreadParticipant = () => {
    var profile;
    return client.execute(() => {
        var threadInfo = document.getElementsByClassName('scrollable')[1];
        var people = threadInfo.getElementsByTagName('h3')[0].nextSibling.nextSibling.nextSibling.getElementsByTagName('a');
        var links = [];
        for(var p = 0; p < people.length; p++) {
            people[p].setAttribute('target', '_self');
            links[links.length] = people[p].getAttribute('href');
        }
        return links;
    })
    .then(r => client.click('a[href*="' + r.value + '"]'))
    .pause(1000)
    .then(() => Promise.all([
        client.getText('h1'),
        client.getText('#intro_container_id li'),
        client.getUrl(),
    ]))
    .then(r => (profile = ({
        name: r[0],
        description: r[1],
        url: r[2],
    })))
    .back()
    .pause(2000)
    .then(() => {console.log(profile); return profile})
};

var switchToParticipantThread = (i) => {
    var part;
    return client.execute(() => {
        var threadInfo = document.getElementsByClassName('scrollable')[1];
        var peopleSection = threadInfo.getElementsByTagName('h3')[0].nextSibling.nextSibling.nextSibling
        var people = peopleSection.getElementsByTagName('li');
        return people;
    })
    .then(r => client.elementIdClick(r.value[i].ELEMENT))
    .pause(1000)
    .then(() => getThreadParticipant())
    .then(p => part = p)
    .back()
    .pause(2000)
    .then(() => part)
};

var getThreadParticipants = (thread) => {
    var readFacebookProfileInfo;
    return client.then(() => importer.interpretAll(['scrape facebook profile']))
    .then(r => r[0].runFresherInNewContext({client}))
    .then(r => readFacebookProfileInfo = r)
    .execute(() => {
        var threadInfo = document.getElementsByClassName('scrollable')[1];
        var peopleSection = threadInfo.getElementsByTagName('h3')[0].nextSibling.nextSibling.nextSibling
        var people = peopleSection.getElementsByTagName('li');
        return peopleSection.getElementsByTagName('h4')[0].innerText.trim() == 'People'
            ? people.length : 0;
    })
    .then(r => {
        var result = [];
        for(var i = 1; i < r.value; i++) {
            result[result.length] = (i =>
                resolve => switchToParticipantThread(i)
                .then(r => resolve(r))
                                     )(i);
        }
        if(r.value === 0) {
            result[result.length] = resolve => getThreadParticipant()
                .then(r => resolve(r))
        }
        return importer.runAllPromises(result);
    })
};

var scrollFacebookMessages = (messages) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
    .execute(() => {
        document.getElementsByClassName('scrollable')[2]
            .scrollTop -= 10000;
    })
    .pause(2000)
    .then(() => readFacebookMessages(messages));
};

var readFacebookMessages = (messages) => {
    return client
    .execute(() => {
        var result = [];
        var groups = document.getElementsByClassName('scrollable')[2];
        var times = groups.getElementsByTagName('h4');
        for(var t = 0; t < times.length; t++) {
            var bubble = times[t];
            while((bubble = bubble.nextSibling) && bubble.nodeName == 'DIV') {
                var from = bubble.getElementsByTagName('h5')[0];
                result[result.length] = {
                    from: typeof from !== 'undefined' ? from.textContent : '',
                    time: times[t].innerText,
                    message: bubble.innerText
                };
            }
        }
        return result;
    })
    .then(r => {
        var newMessages = r.value
            .filter(e => messages
                .filter(m => m.message == e.message).length === 0);
        messages = newMessages.concat(messages);
        return newMessages.length > 0
                ? scrollFacebookMessages(messages)
                : Promise.resolve(messages);
    })
};

var readFacebookThread = (thread, messages) => {
    var participants;
    console.log(thread);
    return client.getUrl().then(url => url.indexOf(thread) == -1
        ? client.url(thread)
        : client)
    // get participants from topcard
    .pause(1000)
    .then(() => getThreadParticipants(thread))
    .then(p => participants = p)
    
    // TODO: save to contacts
    .then(() => readFacebookMessages(messages || []))
    .then(messages => ({
        thread: thread,
        participants: participants || [],
        messages: messages.map(m => {
            var newTime = chrono.parseDate(m.time);
            newTime.setHours(newTime.getHours()-(new Date()).getTimezoneOffset()/60)
            return Object.assign(m, {time: newTime})
        })
    }))
    .catch(e => console.log(e));
};
module.exports = readFacebookThread;
readFacebookThread;



[Function: readFacebookThread]

In [1]:
var importer = require('../Core');
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var readThread = (thread) => {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop();
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    var messages;
    try { messages = require(file) }
    catch (e) { messages = [] }
    return readFacebookThread(thread, messages)
    .then(t => {
        var filename = project + '/' 
            + t.thread.replace(/^\/|\/$/ig, '').split('/').pop()
            + '-' + t.participants
            .map(p => p.name).join('')
            .replace(/[^a-z]/ig, '_') + '.json';
        fs.writeFileSync(filename, JSON.stringify(t, null, 4));
        return t;
    });
};

var listThreads = () => {
    return runSeleniumCell('download facebook messages')
    .then(listFacebookThreads => {
        return listFacebookThreads(threads);
    })
    // list all message threads
    .then(r => fs.writeFileSync(
        project + '/facebook-threads.json',
        JSON.stringify(r, null, 4)))
};

$$.async()
var threads, readFacebookThread;
var runSeleniumCell;

importer.interpretAll(['selenium cell'])
.then(r => {
    runSeleniumCell = r[0].runInNewContext();
    return runSeleniumCell('log in facebook');
})
.then(loginFacebook => loginFacebook())
.then(() => {
    try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
    catch (e) { threads = []; }
    return runSeleniumCell('messages in facebook thread');
})
//.then(() => listThreads())
.then(func => {
    readFacebookThread = func;
    return importer.runAllPromises(threads.slice(0, 1)
        .map(t => ((resolve) => readThread(t).then((r) => resolve(r)))));
})
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
https://www.facebook.com/messages/t/100010177366596
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
{ name: [ 'Facebook', 'Urszula Adamczyk' ],
  description: [ 'Lives in Kielce, Poland', 'From Kielce, Poland' ],
  url: 'https://www.facebook.com/profile.php?id=100010177366596' }


[ { thread: 'https://www.facebook.com/messages/t/100010177366596',
    participants: [ [Object] ],
    messages: 
     [ [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object],
       [Object] ] } ]